In [23]:
import pandas as pd
from EDA import df1
import numpy as np
from eda_helper import rows_to_del_with_index
from eda_helper import createExcel
from sklearn.metrics.pairwise import cosine_similarity
from recomm_sys_v1_helper import top_merch
from recomm_sys_v1_helper import merchCleanup
from recomm_sys_v1_helper import createCatDF
from recomm_sys_v1_helper import takeInputMerch
from recomm_sys_v1_helper import filter1_count_per_merch
from recomm_sys_v1_helper import filter2_meanamt_per_merch
from recomm_sys_v1_helper import rec_df_merch_list
from recomm_sys_v1_helper import filters
from recomm_sys_v1_helper import sparseMtx
from recomm_sys_v1_helper import recommendationSystem

In [2]:
#Thresholds for filtering results from the recommendation engine
Threshold_P = 0.1
Threshold_C = 0.7
Threshold_Cosine=0.6
Threshold_Num_Visit = 6
Threshold_Num_MeanAmt = 10

In [3]:
df1_rec=df1 #creating copy of dataframe for recommendation notebook
#df1_rec.info() #52262 entries - datetime indexed

In [4]:
df1_rec=df1_rec.reset_index() #changing index to int
df1_rec.rename(columns={"simple_description": "merchant"}, inplace=True) #renaming column 'simple_description' to 'merchant'

#df1_rec.head()

### Cleaning some obvious "oops" in merchant (aka simple_description) field like the word pending or visa infront of merchant names etc.

In [6]:
rest_list=[]
for merch in df1_rec.merchant:
    if ((merch[0:8]=='*pending')):
        rest_list.append(merch[30:])
    elif (merch[0:7]=='pending'):
        rest_list.append(merch[23:])
    elif (merch[0:2]=='p '):
        rest_list.append(merch[11:25])
    elif (merch[0:2]=='pp'):
        rest_list.append(merch[3:])
    elif (merch[0:2]=='a '):
        rest_list.append(merch[7:])
    elif (merch[0:8]=='visa pp*'):
        rest_list.append(merch[8:])
    elif (merch[0:7]=='visa sq'):
        rest_list.append(merch[9:])
    elif (merch[0:4]=='visa'):
        rest_list.append(merch[5:26])
    elif (merch[0:2]=='sq'):
        rest_list.append(merch[4:])
    elif (merch[0:2]=='sp'):
        rest_list.append(merch[5:])
    else:
        rest_list.append(merch)
        
#rest_list #visa visa pp* 'sq, visa jefes tacos & tequila 'yonah mountain fridas mexican zucca bar ziggy tandur indi, not just coffe, winot coffee, christ cafe
sorted(rest_list)
df1_rec.reset_index(inplace=True, drop=True)
df1_rec['merchant']=rest_list

In [7]:
#df1_rec.info() #52262 entries

#########################################################################################################################

## Goal # 1: Collaborative Filtering. 
Question for Ana: is it content based filtering or collaborative?

## Recommending merchants based on other similar merchants. 

To implement above scenario, I need to first figure out the merchants in the data. Simple_description (merchant field) has 'not so clear' text and I can extract the "merchant" information from it but since it's all different length text with different start and end words etc., I will focus on extracting merchants from one category at a time and focusing top 10-15 (per Clair's advise) merchants/category

In [8]:
#creating dictionary with category as Key and categoryid as Value

cat_catid_dict=df1_rec[['categoryid','category']].groupby('category')['categoryid'].first().to_dict()
{k: v for k, v in sorted(cat_catid_dict.items(), key=lambda item: item[1])}


{'Automotive Expenses': 2.0,
 'Child/Dependent Expenses': 4.0,
 'Clothing/Shoes': 5.0,
 'Education': 6.0,
 'Entertainment': 7.0,
 'Gasoline/Fuel': 8.0,
 'Gifts': 9.0,
 'Groceries': 10.0,
 'Healthcare/Medical': 11.0,
 'Home Maintenance': 12.0,
 'Home Improvement': 13.0,
 'Cable/Satellite Services': 15.0,
 'Online Services': 16.0,
 'Personal Care': 20.0,
 'Restaurants/Dining': 22.0,
 'Travel': 23.0,
 'Service Charges/Fees': 24.0,
 'Hobbies': 34.0,
 'Telephone Services': 38.0,
 'Utilities': 39.0,
 'Pets/Pet Care': 42.0,
 'Electronics': 43.0,
 'General Merchandise': 44.0,
 'Office Supplies': 45.0,
 'Advertising': 100.0,
 'Postage and Shipping': 104.0,
 'Dues and Subscriptions': 108.0,
 'Shops': 201.0,
 'Service': 202.0,
 'Food and Drink': 203.0,
 'Healthcare': 204.0,
 'Community': 205.0,
 'Recreation': 206.0}

In [9]:
df1_rec.groupby(['category'])['category'].count().sort_values(ascending=False)

category
Restaurants/Dining          8530
Shops                       6403
General Merchandise         6369
Service                     5266
Groceries                   4720
Travel                      3748
Food and Drink              2958
Clothing/Shoes              1879
Entertainment               1755
Online Services             1575
Gasoline/Fuel               1365
Home Improvement            1121
Healthcare/Medical          1018
Personal Care                659
Electronics                  583
Utilities                    566
Hobbies                      556
Service Charges/Fees         352
Automotive Expenses          348
Telephone Services           317
Cable/Satellite Services     315
Pets/Pet Care                302
Community                    267
Recreation                   234
Education                    223
Postage and Shipping         201
Dues and Subscriptions       176
Home Maintenance             111
Healthcare                   108
Gifts                        105
O

#### The recommendation system code here is only coded for and tested for categories with > 1000 records except for Online Services:

#### Restaurants/Dining, Shops, General Merchandise, Service, Groceries, Travel, Food and Drink, Clothing/Shoes, Entertainment,      Gasoline/Fuel, Home Improvement, Healthcare/Medical

### 1. Selecting  a Category and related Merchant within the Category 
For now I am taking input to a "currentCat" variable to determine which category code needs to be executed.
Default category is "Resturants/Dining"

In [12]:
currentCat=input('Category?') #This will come from outside of this code, I think
if currentCat=='': #if left empty code will run for resturant/dining by default
    currentCatid=22.0
    currentCat='Restaurants/Dining'
else:
    currentCatid=cat_catid_dict[currentCat]
currentCatid

Category?


22.0

In [13]:
#Current top 15 merchants in the selected category
#This cell shows what top15 looks like before deleting some that don't make sense in the next step
print("______________________________________________________________")
print("Showing top merchants from category ---  {}".format(currentCat))
print("______________________________________________________________")
top_merch(df1_rec,currentCatid,15) 

______________________________________________________________
Showing top merchants from category ---  Restaurants/Dining
______________________________________________________________


merchant
starbucks                             398
chipotle mexican grill                255
dunkin' donuts                        255
chick-fil-a                           245
firehouse subs                        221
mcdonald's                            211
sushi aji                             171
brewing *******et                     111
wendy's                               106
morning glory cafe                     74
ssp*rockymoutnain kung fuxx2839 co     73
taco bell                              66
bethel university                      65
christi cafe                           65
mici handcrafted italian               62
Name: merchant, dtype: int64

### 2. Cleaning the merchant data some more. Specific to the category.
To see what was deleted and reason behind it read "Deleted_Merchants_History_Story.txt"

In [14]:
#Deleting all the merchant records from all of the categories that are not needed

deleteList=['ssp*rockymoutnain kung fuxx2839 co', 'brewing *******et', 'bethel university','apple.com/bill',\
            'amazon *******etplace','amazon.com','amazon prime video',\
            'ach transaction - qapital transfer 0007392303', 'ach transaction - qapital invest l ach 0007100028',\
            'direct withdrawal','travel','qoins','venmo','porkbun com portland us', 'otis craft collect lafayette us',\
            'live your life integratedxx5229 us', 'paypal','webflow com', 'visible service llc','nzb.su httpsnzb.su fl',\
            'premier1 supplies xx7622 ia','premier1 supplies xx7622 us','home depot',"fisk's 'farm & home suppl decorah ia",\
            "dbt mcguckin hardwar |mcguckin hardware boulder co us|card nbr: 0853",'flying b bar ranch llc xx06 co',\
            'front range eye health louisville co', 'mini miners pediatric erie us','chpg primary care erie erie co',\
            'mini miners pediatric denerie co','lombard direct']
df1_rec=merchCleanup(df1_rec,deleteList)
#df1_rec.info()

#### 3.Creating DF of all the <u>remaining</u> records of the current chosen cateogry(df1_rec_category)

#### 4.Collecting(input) a merchant from selected category for checking model accuracy (merchant_user_visited)



#### 5. Creating filters using 
    (1) popularity of other similar merchants (count_per_merch)  
    (2) mean amount (dollars) spent by other users at the similar merchants (meanamt_per_merch)

#### 6. Create 
    (1) sparse matrix of all user ids (uid) and merchants (merchants) of the specific category (df1_rec_merch_count_mtx)
    (2) create matrix showing how popular the merchant "merchat_user_visited" was with all users in the specific category (merchant_popularity_count)


In [15]:
df1_rec_category, top_merch_list = rec_df_merch_list(df1_rec,currentCatid, top_num=15)

In [16]:
merchant_user_visited,count_per_merch, meanamt_per_merch = filters(currentCatid,df1_rec_category, top_merch_list)

['starbucks', "dunkin' donuts", 'chipotle mexican grill', 'chick-fil-a', 'firehouse subs', "mcdonald's", 'sushi aji', "wendy's", 'morning glory cafe', 'taco bell', 'christi cafe', 'mici handcrafted italian', 'subway', 'ziggis coffee', 'pret a manger']


In [17]:
df1_rec_merch_count_mtx, merchant_popularity_count = sparseMtx(df1_rec_category,merchant_user_visited)

In [18]:
df1_rec_merch_count_mtx

merchant,chick-fil-a,chipotle mexican grill,christi cafe,dunkin' donuts,firehouse subs,mcdonald's,mici handcrafted italian,morning glory cafe,pret a manger,starbucks,subway,sushi aji,taco bell,wendy's,ziggis coffee
uid,,,,,,,,,,,,,,,
005vcYECEUZiN4cXajuea2lBnCy1,0,0,0,0,0,0,1,1,0,4,0,2,0,0,0
0BvDU2CvJTPwox1OYT70bnWdDUB2,0,0,0,0,0,0,1,1,0,2,0,3,0,0,0
0UIAYEH7qlUI6j1iiIg6XbmJWTG3,51,27,0,0,0,36,0,0,0,17,0,0,12,25,46
1I87ooK6OBNwxmmA31vkOHwA8wB3,22,6,0,0,0,3,0,0,0,1,0,0,2,0,0
1fPMZehrbgWogwknDB4ZKluoVFg2,0,17,0,0,0,0,0,0,0,1,7,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tpgEryFqqtSd5juhC85tim271ut2,0,8,0,0,27,0,5,9,0,10,1,14,0,0,0
vw6j8oVvAfhiLuXYahevZgShRXr2,0,3,0,0,0,0,0,0,0,0,1,0,0,0,0
wrHJZ3ZqRaO9VIPd55VTs6qHOpt2,0,4,0,0,0,0,0,0,0,0,0,0,0,1,0


### 8. Building Model

### Evaluating Similarity Based on Pearson Correlation OR Cosine Similarity Score

#### 1 (a) . Compute Pearson Correlation
Using Pandas 'dataframe.corrwith()' method compute pairwise correlation between rows of two dataframes, df1_rec_merch_count_mtx and 'merchant_popularity_count'. This will produce a series with numbers between 0-1, showing correlation between all the merchants and the merchant_user_visited. Closer the score is to 1, greater is the correlation between the two entities. 

#### 1 (b) . Compute Cosine Similarity 
Using sklearn.metrics.pairwise submodule's function, cosine_similarity, compute similarities between all the merchants. The result is a nxn sparse matrix, where n is total number of merchants in a refined category data. A number closer to 1 shows high similarity and a number close to 0 shows less similarity between the merchants.

#### 2. Filering
Using Thresholds for Pearson Score, along with Thresholds for the filters created before (count_per_merch and meanamt_per_merch), filtter out the rows (merchants) that fall within those boundaries. 

#### 3. Recommendations:
        (1) Recommend top 3 merchants that are most similar to the current merchant.

In [19]:
#Pearson

In [20]:
recommendationSystem(merchant_user_visited,'Pearson',df1_rec_merch_count_mtx,merchant_popularity_count, \
                     count_per_merch, meanamt_per_merch,\
                    Threshold_C, Threshold_P, Threshold_Num_MeanAmt, \
                     Threshold_Num_Visit,top_merch_list)

Other merchants you may like ["wendy's", 'chick-fil-a', 'taco bell']


In [21]:
#Cosine

In [22]:
recommendationSystem(merchant_user_visited,'Cosine',df1_rec_merch_count_mtx,merchant_popularity_count, \
                     count_per_merch, meanamt_per_merch, \
                    Threshold_C, Threshold_P, Threshold_Num_MeanAmt, \
                     Threshold_Num_Visit,top_merch_list)


Other merchants you may like ['starbucks', 'subway', 'taco bell']


The End!